In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
data_path = '../data/raw/Train.csv'

In [ ]:
df = pd.read_csv(data_path)
df = df.set_index('ID')

In [ ]:
list_cols = ['LandPreparationMethod', 'NursDetFactor', 'TransDetFactor', 'OrgFertilizers', 'CropbasalFerts', 'FirstTopDressFert']

cat_cols = ['District', 'Block', 'CropEstMethod', 'TransplantingIrrigationSource', 'TransplantingIrrigationPowerSource', 
            'PCropSolidOrgFertAppMethod', 'MineralFertAppMethod', 'MineralFertAppMethod.1', 'Harv_method', 'Threshing_method', 'Stubble_use']

num_cols = ['CultLand', 'CropCultLand', 'CropTillageDepth', 'SeedlingsPerPit', 'TransplantingIrrigationHours', 'TransIrriCost', 
            'StandingWater', 'Ganaura', 'CropOrgFYM', 'NoFertilizerAppln', 'BasalDAP', 'BasalUrea', '1tdUrea', '1appDaysUrea', 
            '2tdUrea', '2appDaysUrea', 'Harv_hand_rent', 'Residue_length', 'Residue_perc', 'Acre', 'Yield']

date_cols = ['CropTillageDate', 'RcNursEstDate', 'SeedingSowingTransplanting', 'Harv_date', 'Threshing_date']

In [ ]:
columns = list_cols + cat_cols + num_cols + date_cols

In [ ]:
len(columns) == len(df.columns)

## Process list columns

In [ ]:
for col in list_cols:
    split_col = df[col].str.split().explode()
    split_col = pd.get_dummies(split_col, prefix=col, prefix_sep='')
    split_col = split_col.astype(int).groupby(level=0).max()
    df = df.join(split_col)
    df = df.drop(columns=[col])

## Process post list columns

### FYM

In [ ]:
# Assumption: CropOrgFYM correlated with OrgFertilizersFYM
df_0 = df[['CropOrgFYM', 'OrgFertilizersFYM']][df['OrgFertilizersFYM'] == 0]
df_1 = df[['CropOrgFYM', 'OrgFertilizersFYM']][df['OrgFertilizersFYM'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

### Ganaura

In [ ]:
# Assumption: Ganaura correlated with OrgFertilizersGanaura
df_0 = df[['Ganaura', 'OrgFertilizersGanaura']][df['OrgFertilizersGanaura'] == 0]
df_1 = df[['Ganaura', 'OrgFertilizersGanaura']][df['OrgFertilizersGanaura'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

### DAP

In [ ]:
# Assumption: BasalDAP correlated with CropbasalFertsDAP
df_0 = df[['BasalDAP', 'CropbasalFertsDAP']][df['CropbasalFertsDAP'] == 0]
df_1 = df[['BasalDAP', 'CropbasalFertsDAP']][df['CropbasalFertsDAP'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

df_0 = df[['BasalDAP', 'FirstTopDressFertDAP']][df['FirstTopDressFertDAP'] == 0]
df_1 = df[['BasalDAP', 'FirstTopDressFertDAP']][df['FirstTopDressFertDAP'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

### Urea

In [ ]:
# Assumption: BasalUrea correlated with CropbasalFertsUrea and 1tdUrea correlated with FirstTopDressFertUrea
df[['BasalUrea', '1tdUrea', 'CropbasalFertsUrea', 'FirstTopDressFertUrea']]

df_0 = df[['BasalUrea', 'CropbasalFertsUrea']][df['CropbasalFertsUrea'] == 0]
df_1 = df[['BasalUrea', 'CropbasalFertsUrea']][df['CropbasalFertsUrea'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

df_0 = df[['1tdUrea', 'FirstTopDressFertUrea']][df['FirstTopDressFertUrea'] == 0]
df_1 = df[['1tdUrea', 'FirstTopDressFertUrea']][df['FirstTopDressFertUrea'] == 1]

print(df_0.isnull().sum(), '\n', df_0.isnull().sum() / len(df_0), '\n')
print(df_1.isnull().sum(), '\n', df_1.isnull().sum() / len(df_1))

### Columns processing

In [ ]:
corr_list_cols = [('CropOrgFYM', 'OrgFertilizersFYM'), ('Ganaura', 'OrgFertilizersGanaura'), ('BasalDAP', 'CropbasalFertsDAP'), ('BasalUrea', 'CropbasalFertsUrea'), ('1tdUrea', 'FirstTopDressFertUrea')]

In [ ]:
for col1, col2 in corr_list_cols:
    df.loc[df[col2] == 0, col1] = 0
    df = df.drop(columns=[col2])

## Process date columns

In [ ]:
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
    df[f'{col}Year'] = df[col].dt.year.astype('string')
    df[f'{col}DayOfYear'] = df[col].dt.dayofyear
    df[f'{col}DayOfYearSin'] = np.sin(2 * np.pi * df[f'{col}DayOfYear'] / 365)
    df[f'{col}DayOfYearCos'] = np.cos(2 * np.pi * df[f'{col}DayOfYear'] / 365)
    cat_cols.append(f'{col}Year')
    df = df.drop(columns=[col, f'{col}DayOfYear'])

## One hot encoding

In [ ]:
for col in cat_cols:
    df[col] = df[col].fillna('Unknown')
    ohe_col = pd.get_dummies(df[col], prefix=col, prefix_sep='')
    ohe_col = ohe_col.astype(int).groupby(level=0).max()
    df = df.join(ohe_col)
    df = df.drop(columns=[col])

## Missing values

In [ ]:
# delete columns
missing_column_thr = 50
missing_column = df.isnull().sum() / len(df) * 100 > missing_column_thr
to_delete_cols = missing_column[missing_column].index.tolist()
df = df.drop(columns=to_delete_cols)

In [ ]:
fill_mode = 'median' # or 'median'

missing_column = df.isnull().sum() / len(df) * 100 > 0
to_fill_cols = missing_column[missing_column].index.tolist()

to_fill_values = []
for col in to_fill_cols:
    if fill_mode == 'mean':
        value = df[col].mean()
        df[col] = df[col].fillna(value)
    elif fill_mode == 'median':
        value = df[col].median()
        df[col] = df[col].fillna(value)
    else:
        raise ValueError('Unknown filling mode')

    to_fill_values.append({col: value})

In [ ]:
unique_value_cols = []

for col in df.columns:
    num_unique_values = len(df[col].unique())
    
    if num_unique_values == 1:
        df = df.drop(columns=[col])
        unique_value_cols.append(col)

In [ ]:
plt.figure(figsize=(5, 50))
df['yield_acre'] = df['Yield'] / df['Acre']
heatmap = sns.heatmap(df.corr()[['yield_acre']].sort_values(by='yield_acre', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')

In [ ]:
plt.figure(figsize=(5, 50))
df['YieldByAcre'] = df['Yield'] / df['Acre']
heatmap = sns.heatmap(df.corr()[['Yield', 'YieldByAcre']].sort_values(by='Yield', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')